# Consume an Object Detection gRPC service

In [6]:
#!pip install tensorflow-serving-api==1.12

In [7]:
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
import grpc
from PIL import Image
import numpy as np
import requests
import cv2

In [8]:
def load_image_into_numpy_array(input_image):
    image = Image.open(input_image)
    (im_width, im_height) = image.size
    image_arr = np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)
    image.close()
    return image_arr

def load_input_tensor(input_image):
    image_np = load_image_into_numpy_array(input_image)
    image_np_expanded = np.expand_dims(image_np, axis=0).astype(np.uint8)
    tensor = tf.contrib.util.make_tensor_proto(image_np_expanded)
    return tensor

In [9]:
SERVICE_ENDPOINT = 'localhost:8500'

INPUT_IMAGE_FILENAME = "../../media/test_image.jpeg"

channel = grpc.insecure_channel(SERVICE_ENDPOINT)

# Send request
# See prediction_service.proto for gRPC request/response details.
request = predict_pb2.PredictRequest()
request.model_spec.name = 'faster_rcnn_resnet50_coco'
request.model_spec.signature_name = 'serving_default'
input_tensor = load_input_tensor(INPUT_IMAGE_FILENAME)
request.inputs['inputs'].CopyFrom(input_tensor)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
result = stub.Predict(request, 10.0)  # 10 secs timeout

In [10]:
import matplotlib.pyplot as plt

image_np = load_image_into_numpy_array(INPUT_IMAGE_FILENAME)
plt.imshow(image_np)
plt.show()
image_np.shape

<Figure size 640x480 with 1 Axes>

(177, 284, 3)

In [11]:


output_dict = {}
output_dict["classes"] = np.squeeze( result.outputs["detection_classes"].float_val).astype(np.uint8)
output_dict["detection_boxes"] = np.reshape(result.outputs["detection_boxes"].float_val, (-1, 4))
output_dict["detection_scores"] = np.squeeze(result.outputs["detection_scores"].float_val)
output_dict


for i in range(len(output_dict["detection_scores"])):
    prob_ci = output_dict["detection_scores"][i]
    if prob_ci > 0.5: 
        print("Class: ", output_dict["classes"][i])
        print("Score: ", output_dict["detection_scores"][i])
        box =  output_dict["detection_boxes"][i]
        box[0] = box[0] * image_np.shape[0]
        box[1] = box[1] * image_np.shape[1]
        box[2] = box[2] * image_np.shape[0]
        box[3] = box[3] * image_np.shape[1]
        print("Box: ", box)

#for b in output_dict["detection_boxes"]:
#    print(b)
    


Class:  1
Score:  0.9993390440940857
Box:  [ 93.25476837 195.30640316 153.74656963 215.07755589]
Class:  1
Score:  0.9967325925827026
Box:  [ 95.84371215  96.94385397 137.81718618 110.24471617]
Class:  1
Score:  0.9931192398071289
Box:  [ 89.22938633  47.4890613  145.92815924  64.66537666]
Class:  1
Score:  0.9870758652687073
Box:  [105.76085955  64.38819373 140.46374363  87.97848535]
Class:  1
Score:  0.9396260976791382
Box:  [124.99993265 106.06578708 144.10271001 129.12431133]
Class:  1
Score:  0.7845249772071838
Box:  [124.29547602 157.39174867 144.10973632 164.96018267]


## REST API

Check model status and Predict signature.

In [12]:
HOSTNAME="localhost"
MODEL="faster_rcnn_resnet50_coco"
REST_API_ENDPOINT='http://%s:8501/v1/models/%s' % (HOSTNAME, MODEL)

r = requests.get(REST_API_ENDPOINT)
r.json()

{'model_version_status': [{'state': 'AVAILABLE',
   'status': {'error_code': 'OK', 'error_message': ''},
   'version': '20180128'}]}

In [13]:
r = requests.get(REST_API_ENDPOINT+"/metadata")
r.json()

{'metadata': {'signature_def': {'signature_def': {'serving_default': {'inputs': {'inputs': {'dtype': 'DT_UINT8',
       'name': 'image_tensor:0',
       'tensor_shape': {'dim': [{'name': '', 'size': '-1'},
         {'name': '', 'size': '-1'},
         {'name': '', 'size': '-1'},
         {'name': '', 'size': '3'}],
        'unknown_rank': False}}},
     'method_name': 'tensorflow/serving/predict',
     'outputs': {'detection_boxes': {'dtype': 'DT_FLOAT',
       'name': 'detection_boxes:0',
       'tensor_shape': {'dim': [{'name': '', 'size': '-1'},
         {'name': '', 'size': '100'},
         {'name': '', 'size': '4'}],
        'unknown_rank': False}},
      'detection_classes': {'dtype': 'DT_FLOAT',
       'name': 'detection_classes:0',
       'tensor_shape': {'dim': [{'name': '', 'size': '-1'},
         {'name': '', 'size': '100'}],
        'unknown_rank': False}},
      'detection_scores': {'dtype': 'DT_FLOAT',
       'name': 'detection_scores:0',
       'tensor_shape': {'dim': [{